In [48]:
from bs4 import BeautifulSoup
from urllib import request
import pytz
from datetime import datetime
import pandas as pd
import os

In [70]:
BASE_URL = "http://www.espn.com/nfl/lines"
with request.urlopen(BASE_URL) as response:
   html = response.read()   
soup = BeautifulSoup(html, "lxml")  
table = soup.find("table", { "class" : "tablehead" })


In [71]:
tz = tz = pytz.timezone('America/New_York')

In [95]:
outfile = open("Lines.txt", "a")
for row in table.findAll("tr"):
    cells = row.findAll("td")
    c0 = cells[0].find(text=True)
    if len(cells) == 1 and len(c0) > 10:
        game = c0
        AwayTeam = game[0:game.index(' at')]
        HomeTeam = game[game.index(' at')+4:game.index(' -')] 
        #print(HomeTeam,AwayTeam) 
    elif len(cells) == 9:
        source = cells[0].find(text=True)
        Line = cells[1].find(text=True)
        #AwayTeam = cells[3].find(text=True)
        #AwayTeam = AwayTeam[:AwayTeam.index(':')]
        OverUnder = cells[5].find(text=True)
        OverUnder = OverUnder[:OverUnder.index(" ")]
        
        if float(Line) > 0:
            Favorite = HomeTeam
            Underdog = AwayTeam
        elif float(Line):
            Favorite = AwayTeam
            Underdog = HomeTeam
        else:
            Favorite = 'Pickem'
            HomeTeam = 'Pickem'
    elif len(cells) == 2 and cells[1].find(text=True)[:3] == 'o: ':
        #print (datetime.now(tz).isoformat(timespec='minutes'),"|",game,"|",source,"|",abs(float(Line)),"|",
        #       HomeTeam,"|",OverUnder,"|",HomeTeam,"|",AwayTeam,"|",Favorite,"|",Underdog)
        outfile.write(datetime.now(tz).isoformat(timespec='seconds') +"\t"+ game +"\t"+ source +"\t"+ 
        str(abs(float(Line))) +"\t"+ OverUnder + "\t"+ HomeTeam + "\t"+ AwayTeam +'\t'+ Favorite +'\t'+ Underdog + '\n')
outfile.close()